## 赛题描述
本次大赛基于脱敏和采样后的数据信息，预测未来一段时间活跃的用户。

## 数据说明
大赛提供脱敏和采样后用户行为数据，日期信息进行统一编号，第一天编号为 01， 第二天为 02， 以此类推，所有文件中列使用 tab 分割。
一个二分类问题，即判断一个用户是否为活跃用户。

## 数据文件下载地址

链接：https://pan.baidu.com/s/1MhAU0zyMHA5-bUbPUjWRzQ 
提取码：dz9p

## 文件说明
###    app_launch_log.txt：用户登录数据集
    列名            类型          说明                     示例
    user_id         Int           用户唯一标识（脱敏后）   666
    day             String        日期                     01, 02 …30
    
###    user_activity_log.txt：用户行为数据集（对视频点赞，转发等）
    列名            类型          说明                     示例
    user_id         Int           用户唯一标识（脱敏后）   666
    day             String        日期                     01, 02 …30
    page            Int           行为发生的页面。         1
                                  每个数字分别对应
                                  “关注页”、“个人主页”
                                  “发现页”、“同城页”
                                  或“其他页”中的一个   
    video_id        Int           video id（脱敏后）       333
    author_id       Int           作者 id（脱敏后）        999
    action_type     Int           用户行为类型。           1
                                  每个数字分别对应
                                 “播放”、“关注”
                                 “点赞”、“转发”、
                                 “举报”和“减少此类作品”中的一个	

###    user_register_log.txt：用户注册数据集
    列名            类型          说明                     示例
    user_id         Int           用户唯一标识（脱敏后）   666
    register_day    String        日期                     01, 02 … 30
    register_type   Int           来源渠道（脱敏后）       0
    device type     Int           设备类型（脱敏后）       0
    
###    video_create_log.txt：视频创建数据集
    列名            类型           说明                    示例
    user_id         Int            用户唯一标识（脱敏后）  666
    day             String         拍摄日期                01, 02 … 30

In [1]:
import pandas as pd
import numpy as np

In [2]:
# 数据集中使用制表符来进行分割
#注册数据
register = pd.read_csv('user_register_log.txt',sep='\t',names=['user_id','register_day','register_type','device_type'])
#登录数据
launch = pd.read_csv('app_launch_log.txt',sep='\t',names=['user_id','launch_day'])
#创建视频数据
create = pd.read_csv('video_create_log.txt',sep='\t',names=['user_id','create_day'])
#行为数据
activity = pd.read_csv('user_activity_log.txt',sep='\t',names=['user_id','act_day','page','video_id','author_id','act_type'])

In [6]:
print(register.shape)
print(launch.shape)
print(create.shape)
print(activity.shape)

(51709, 4)
(251943, 2)
(35151, 2)
(20607228, 6)


### 用户注册数据
    列名            类型          说明                     示例
    user_id         Int           用户唯一标识（脱敏后）   666
    register_day    String        日期                     01, 02 … 30
    register_type   Int           来源渠道（脱敏后）       0
    device type     Int           设备类型（脱敏后）       0

In [7]:
register.head()

,user_id,register_day,register_type,device_type
0,744025,1,1,283
1,1270299,1,1,259
2,571220,1,1,2
3,1308501,1,0,23
4,745554,1,2,0


### 用户登录数据

    列名            类型          说明                     示例
    user_id         Int           用户唯一标识（脱敏后）   666
    day             String        日期                     01, 02 …30

In [11]:
#这个用户在哪些天登录了（每个用户可能不止一条）
launch.head()

,user_id,launch_day
0,383135,1
1,330986,4
2,330986,9
3,330986,11
4,330986,12


### 创建视频数据
    列名            类型           说明                    示例
    user_id         Int            用户唯一标识（脱敏后）  666
    day             String         拍摄日期                01, 02 … 30

In [12]:
#按照创建视频的时间记录，每个用户可能在同一天创建不止一个视频
create.head()

,user_id,create_day
0,720497,1
1,720497,1
2,720497,1
3,1075211,6
4,1075211,12


### 行为数据集    
    列名            类型          说明                     示例
    user_id         Int           用户唯一标识（脱敏后）   666
    day             String        日期                     01, 02 …30
    page            Int           行为发生的页面。         1
                                  每个数字分别对应
                                  “关注页”、“个人主页”
                                  “发现页”、“同城页”
                                  或“其他页”中的一个   
    video_id        Int           video id（脱敏后）       333
    author_id       Int           作者 id（脱敏后）        999
    action_type     Int           用户行为类型。           1
                                  每个数字分别对应
                                 “播放”、“关注”
                                 “点赞”、“转发”、
                                 “举报”和“减少此类作品”中的一个

In [10]:
activity.head()

,user_id,act_day,page,video_id,author_id,act_type
0,1062323,22,3,2877472,880271,0
1,639898,17,3,740662,210200,0
2,1260200,5,3,3332414,162866,0
3,817201,22,3,1129617,530246,0
4,817201,23,3,1129617,530246,0


    目标：预测第n+7天用户是否活跃
    模型：RNN
    Feature：  t1   ->  t2     ->   ...  ->   tn 天的数据
                |        |                     |   
    Label:     t(1+7)   t(2+7)       ...      t(n+7)天的活跃情况（是否登录）